In [34]:
import numpy as np
import torchvision.datasets as datasets
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import torch.utils.data as data

In [26]:
#use gpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [27]:
#import cifar10 dataset
train_set = datasets.CIFAR10(root = "Data", train = True, download = True, transform = ToTensor())
test_set= datasets.CIFAR10(root = "Data",train = False, download = True, transform = ToTensor() )

Files already downloaded and verified
Files already downloaded and verified


In [51]:
#design the neural network

def makethenet(printtoggle=False):
    
    class cifar_cnn(nn.Module):

        def __init__(self,printtoggle):
            #we call super here so as to inherit from nn.Moduule, which is the base class for  Neural networks
            super().__init__()

            #print toggle
            self.print = printtoggle

            ## --- feature map layers --- ##

            # 1. convolution layer
            self.conv1 = nn.Conv2d(3,64,3, padding = 1)   #returns a convoluting object which is later used for convolution
            self.bnorm1 = nn.BatchNorm2d(64)

            #2. convolution layer
            self.conv2 = nn.Conv2d(64,128,3)              #returns a convoluting object
            self.bnorm2 = nn.BatchNorm2d(128)

            #2. convolution layer
            self.conv3 = nn.Conv2d(128,256,3)              #returns a convoluting object
            self.bnorm3 = nn.BatchNorm2d(256)

            ## -- Linear layers -- ##
            self.fc1 = nn.Linear(2*2*256,256)
            self.fc2 = nn.Linear(256,64)
            self.fc3 = nn.Linear(64,10)

        def forward(self,x):

            if self.print: print(f'Input: {list(x.shape)}')

            ## --- first block(includes a set of conv, pooling and nn layers) --- ##

            #apply max pooling after convolution
            x = F.max_pool2d(self.conv1(x),2)
            #apply leaky relu after batch normalising
            x = F.leaky_relu(self.bnorm1(x))
            if self.print :print(f'Second CPR Block: {list(x.shape)}')

            ## --- second block: convolution -> maxpool -> batchnorm -> relu --- ##
            x = F.max_pool2d(self.conv2(x),2)
            x = F.leaky_relu(self.bnorm2(x))
            if self.print: print(f'Second CPR block: {list(x.shape)}')

            ## --- third block: convolution -> maxpool -> batchnorm -> relu --- ##
            x = F.max_pool2d(self.conv3(x),2)
            x = F.leaky_relu(self.bnorm3(x))
            if self.print: print(f'Third CPR block: {list(x.shape)}')


            # --- we need to change the shape so as to fit into the linear layers --- #
            nUnits = x.shape.numel()/x.shape[0]          #numel() returns the total no. of elements in tensor        
            x = x.view(-1,int(nUnits))                   #view() return a tensor with same data but different shape


            # --- pass through the linear layer --- #
            x = F.leaky_relu(self.fc1(x))
            x = F.dropout(x,p=0.5,training=self.training)
            x = F.leaky_relu(self.fc2(x))
            x = F.dropout(x,p=.5,training=self.training) # training=self.training means to turn off during eval mode
            x = self.fc3(x)
            if self.print: print(f'Final output: {list(x.shape)}')

            return x
    
    #create the model instance
    net = cifar_cnn(printtoggle)
    
    #loss function 
    lossfun = nn.CrossEntropyLoss()
    
    #optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr = 0.001, weight_decay = 1e-5)
    
    
    return net,lossfun,optimizer     
        

In [30]:
train_loader = data.DataLoader(train_set,shuffle = True, batch_size = 64)
test_loader = data.DataLoader(test_set,batch_size = 64, shuffle = True)

In [52]:
#test the model with one batch
net,lossfun,optimizer = makethenet(True)

X,y = next(iter(train_loader))
print(f'type of X: {type(X)}')
yHat = net(X)

# check size of output
print('\nOutput size:')
print(yHat.shape)

# now compute the loss
loss = lossfun(yHat,torch.squeeze(y))
print(' ')
print('Loss:')
print(loss)

type of X: <class 'torch.Tensor'>
Input: [64, 3, 32, 32]
Second CPR Block: [64, 64, 16, 16]
Second CPR block: [64, 128, 7, 7]
Third CPR block: [64, 256, 2, 2]
Final output: [64, 10]

Output size:
torch.Size([64, 10])
 
Loss:
tensor(2.3112, grad_fn=<NllLossBackward0>)


In [ ]:
# training the model

def function2train():
    
    #number of epochs
    num_pochs = 10
    
    #create a new model
    net, lossfun, optimizer = makethenet()
    
    #send the model to the GPU
    net.to(device)
    
    #initialize loses and accuracy
    trainloss = torch.zeros(num_pochs)
    trainacc = torch.zeros(num_pochs)
    
    #loop for each epoch
    for epoch in range(num_pochs):
        
        # loop over training data batches
        net.train() #switch to train mode
        batchloss = []
        batchacc = []
        for X,y in train_loader:
            
            #push data to GPU
            X = X.to(device)
            y = y.to(device)
            
            
            #forwardpass and loss
            yhat = net(X)
            loss = lossfun(yhat,y)
            
            #backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            #loss and accuracy from this batch
            batchloss.append(loss.item())
            batchacc.append(torch.mean((torch.argmax(yhat,axis =1)==y).float()).item())
            
        # get average losses and accuracies across the batches
        trainloss[epoch] = np.mean(batchloss)
        trainacc[epoch] = 100*np.mean(batchacc)

        print(trainloss)
        print(trainacc)
        
        return trainloss,trainacc,net
    
y = function2train()